In [ ]:
%pip install gadm hdx-python-api
%pip install geopandas --upgrade
%pip install pyomo
%pip install highspy
%pip install chart_studio
%pip install h5netcdf

In [ ]:
import folium as fl
import pandas as pd
import geopandas as gpd
from hdx.api.configuration import Configuration
from hdx.data.resource import Resource
import urllib.request
import requests
import json
import requests
import itertools

from shapely.geometry import Polygon,MultiPolygon
from shapely.ops import unary_union

from gadm import GADMDownloader
import numpy as np

import xarray as xr
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py

import matplotlib.pyplot as plt

In [ ]:
# Initialize the GADMDownloader with the specified version (in this case, version 4.0)
downloader = GADMDownloader(version="4.0")

# Define the country name for which you want to retrieve administrative boundary data
country_name = "Serbia"

# Specify the administrative level you are interested in (e.g., 1 for districts or provinces)
ad_level = 0

# Retrieve the geospatial data for the selected country and administrative level
gdf = downloader.get_shape_data_by_country_name(country_name=country_name, ad_level=ad_level)

# Display the first 2 rows of the obtained geospatial data for a quick preview
gdf.head(2)


In [ ]:
# Create a Folium map (m) with an initial zoom level of 10 and using OpenStreetMap tiles as the basemap
m = fl.Map(location=[44.0165, 21.0059], zoom_start=8, tiles="OpenStreetMap")

# Iterate through each row in the geospatial data (gdf) representing administrative boundaries
for _, r in gdf.iterrows():
    # Simplify the geometry of the current boundary with a specified tolerance
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.00001)

    # Convert the simplified geometry to JSON format
    geo_j = sim_geo.to_json()

    # Create a GeoJson layer from the JSON geometry, and style it with an orange fill color
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})

    # Add a popup with the NAME_1 attribute (administrative region name) to the GeoJson layer
    fl.Popup(r["COUNTRY"]).add_to(geo_j)

    # Add the styled GeoJson layer to the Folium map (m)
    geo_j.add_to(m)

# Display the Folium map (m) with the administrative boundaries and popups
#m


In [ ]:
selected_gadm = gdf

In [ ]:
population = pd.read_csv('ppp_SRB_2020_1km_Aggregated.csv')

In [ ]:
print('Total Population:',round(population['Z'].sum()/1000000,2),'million')
population = gpd.GeoDataFrame(
    population, geometry=gpd.points_from_xy(population.X, population.Y)
)

population = population.set_crs(selected_gadm.crs)

In [ ]:
# Perform a spatial join to find population within the selected administrative boundary
population_aoi = gpd.sjoin(population, selected_gadm)[['X','Y','Z','geometry']]
population_aoi.columns = ['Longitude','Latitude','Population','geometry']
population_aoi = population_aoi.reset_index().reset_index()
del population_aoi['index']
population_aoi = population_aoi.rename(columns={'level_0':'ID'})
print('Total Population:',round(population_aoi['Population'].sum()))

### This code segment retrieves and analyzes healthcare facility data (hospitals and clinics) in Serbia within a specified area of interest (AOI). Here's a brief summary of what it does:

- It uses the Overpass API to query OpenStreetMap data for hospitals in Timor-Leste, retrieves the data in JSON format, and converts it into a DataFrame (`df_hospitals`).

- It extracts relevant information, such as the hospital's name, latitude, and longitude, from the OpenStreetMap data.

- Similarly, it queries OpenStreetMap data for clinics in Timor-Leste, retrieves the data, and processes it into a DataFrame (`df_clinics`), extracting relevant information.

- The code then combines the hospital and clinic data into a single GeoDataFrame (`df_health_osm`) and converts latitude and longitude coordinates into a geometry column.

- It prints the number of hospitals and clinics extracted from the data.

- Finally, it performs a spatial join to determine how many hospitals and clinics fall within the specified administrative region of interest (AOI) and prints the result.

This code segment is a critical step in assessing healthcare accessibility in a specific region of Timor-Leste, as it identifies and quantifies the healthcare facilities within the chosen area.

In [ ]:
%%time

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="RS"];
(node["amenity"="hospital"](area);
 way["amenity"="hospital"](area);
 rel["amenity"="hospital"](area);
);
out center;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()

df_hospitals = pd.DataFrame(data['elements'])

df_hospitals['name'] = df_hospitals['tags'].apply(lambda x:x['name'] if 'name' in list(x.keys()) else None)

df_hospitals = df_hospitals[['id','lat','lon','name']].drop_duplicates()

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="RS"];
(node["amenity"="clinic"](area);
 way["amenity"="clinic"](area);
 rel["amenity"="clinic"](area);
);
out center;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()

df_clinics = pd.DataFrame(data['elements'])
df_clinics['name'] = df_clinics['tags'].apply(lambda x:x['name'] if 'name' in list(x.keys()) else None)
df_clinics['amenity'] = df_clinics['tags'].apply(lambda x:x['healthcare'] if 'healthcare' in list(x.keys()) else None)

df_clinics = df_clinics[['id','lat','lon','name','amenity']].drop_duplicates()

df_health_osm = pd.concat([df_hospitals,df_clinics])
df_health_osm = gpd.GeoDataFrame(df_health_osm, geometry=gpd.points_from_xy(df_health_osm.lon, df_health_osm.lat))
df_health_osm = df_health_osm[['id','name','geometry']]

print('Number of hospitals and clinics extracted:',len(df_health_osm))
df_health_osm = df_health_osm.set_crs(selected_gadm.crs)
selected_hosp = gpd.sjoin(df_health_osm, selected_gadm, predicate='within')
print('Number of hospitals and clinics in Serbia',len(selected_hosp))

In [ ]:
len(selected_hosp)

In [ ]:
selected_hosp.head()

In [ ]:
def get_isochrone_osm (each_hosp,travel_time_secs):
  body = {"locations":[[each_hosp.x,each_hosp.y]],"range":[travel_time_secs],"range_type":'time'}
  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': '5b3ce3597851110001cf62487e439dfd168049a694bf909262583588',
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post('https://api.openrouteservice.org/v2/isochrones/driving-car', json=body, headers=headers)
  if(call.status_code==200):
    geom = (json.loads(call.text)['features'][0]['geometry'])
    polygon_geom = Polygon(geom['coordinates'][0])
    return polygon_geom
  else:
    return None

In [ ]:
selected_hosp['cachment_area_osm'] = selected_hosp['geometry'].apply(get_isochrone_osm,travel_time_secs=3600)


In [ ]:
selected_hosp.head()

In [ ]:
quartile_labels = [0.1, 0.25, 0.5, 1.0]
population_aoi['opacity'] = pd.qcut(population_aoi['Population'], 4, labels=quartile_labels)

In [ ]:
selected_hosp['cachment_area'] = selected_hosp['cachment_area_osm']

In [ ]:
def get_pop_count(cachment,pop_data):
  if(cachment!=None):
    pop_access = pop_data[pop_data.within(cachment)]
    id_values = (pop_access['ID'].values)
    pop_with_access = (pop_access['Population'].sum().round())
    return id_values,pop_with_access
  else:
    return [None,None]

selected_hosp['id_with_access'], selected_hosp['pop_with_access'] = zip(*selected_hosp['cachment_area'].apply(get_pop_count, pop_data=population_aoi))


In [ ]:
selected_hosp.head(2)

In [ ]:
selected_hosp = selected_hosp.dropna()

In [ ]:
list_ids_access = [ids if ids is not None else [] for ids in selected_hosp['id_with_access']]
list_ids_access = list(itertools.chain.from_iterable(list_ids_access))
pop_with_access = population_aoi[population_aoi['ID'].isin(list_ids_access)]
pop_without_access = population_aoi[~population_aoi['ID'].isin(list_ids_access)]

original_access = round(pop_with_access['Population'].sum()*100/population_aoi['Population'].sum(),2)

print('Population with Access:',round(pop_with_access['Population'].sum()*100/population_aoi['Population'].sum(),2),'%')

In [ ]:
folium_map = fl.Map(location=[44.0165, 21.0059], zoom_start=8, tiles="OpenStreetMap")

geo_adm = fl.GeoJson(data=selected_gadm.iloc[0]['geometry'],style_function=lambda x:{'color': 'orange'})
geo_adm.add_to(folium_map)

for i in range(0,len(selected_hosp)):
    fl.Marker([selected_hosp.iloc[i]['geometry'].y, selected_hosp.iloc[i]['geometry'].x],
                        color='blue',popup=selected_hosp.iloc[i]['name']).add_to(folium_map)

for i in range(0,len(pop_with_access)):
  fl.CircleMarker(
        location=[pop_with_access.iloc[i]['Latitude'], pop_with_access.iloc[i]['Longitude']],
        radius=5,
        color=None,
        fill=True,
        fill_color='green',
        fill_opacity=pop_without_access.iloc[i]['opacity']).add_to(folium_map)

folium_map


In [ ]:
historic_data_path = 'SRB_historical_ERA5-Land_1981-2023_v1.nc'
prediction_data_path = 'SRB_projection_ISIMIP_2015-2100_v1.nc'

In [ ]:
ncdf_data_history = xr.open_dataset(historic_data_path, engine='netcdf4')
ncdf_data_future = xr.open_dataset(prediction_data_path, engine='netcdf4')

In [ ]:
ncdf_data_history = ncdf_data_history.to_dataframe().dropna().reset_index()[['lat','lon','time','pr','sfcwind','tas','tasmax','tasmin']]
ncdf_data_future = ncdf_data_future.to_dataframe().dropna().reset_index()[['lat','lon','time','pr','sfcwind','tas','tasmax','tasmin']]


In [ ]:
ncdf_data_history.head()

In [ ]:
ncdf_geo_data_history = gpd.GeoDataFrame(ncdf_data_history,
                                     geometry=gpd.points_from_xy(ncdf_data_history.lon, ncdf_data_history.lat))

In [ ]:
ncdf_geo_data_history['grid'] = ncdf_geo_data_history['geometry'].buffer(0.05, cap_style = 3)
gridded_data = ncdf_geo_data_history.set_crs('EPSG:4326')

In [ ]:
def data_from_geometry(geometry_file,gridded_data,sel_crs):
    unique_grid = gridded_data['grid'].unique()
    result_grid = []

    x = 0
    for each_grid in unique_grid:
        grid_id = 'grid'+str(x+1)
        for each_val in geometry_file['geometry'].values:
            if (each_val.intersects(each_grid)):
                result_grid.append([each_grid,grid_id])
                x = x+1

    result_df = pd.DataFrame(result_grid)
    result_df.columns = ['grid','grid_id']
    result_gdf = gpd.GeoDataFrame(result_df,geometry='grid')
    result_gdf = result_gdf.set_crs(sel_crs)

    extracted_data = result_gdf.merge(gridded_data,on='grid')
    return extracted_data

In [ ]:
df = pd.DataFrame(
    {'name': ['Natus E&S'],
     'Latitude': [21.69775],
     'Longitude': [43.51355]})

geometry_file = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
extracted_data = data_from_geometry(geometry_file,gridded_data,'EPSG:4326')

In [ ]:
extracted_data